# 은행 계좌 신규, 입금, 출금 처리하기

     1. 은행 계좌만들기
     2. 계좌에 대한 입출금처리하기
     

## 계좌번호 채번하기
    
    계좌번호를 채번을 관리하는 것 하나의 class로 관리

In [49]:
%%writefile bank/AccountNumber.py

# -*- coding: utf-8 -*-
"""
Created on Mon Feb 29 16:58:53 2016

@author: 06411
"""


class Account_Nubmer(object) :
    _Account_No = 0
    @classmethod
    def set_AN(cls) :
        cls._Account_No  +=1
        
    @classmethod
    def get_AN(cls) :
        if not cls._Account_No  :
            cls.set_AN()
        return cls._Account_No 


Overwriting bank/AccountNumber.py


## 수신계좌 만들기

     수신은 예금을 관리하는 계좌이며 신규, 출금, 입금, 거래내역을 관리

In [51]:
%%writefile bank/Account.py
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 29 12:09:56 2016

@author: 06411
"""


import AccountNumber as an

class DepositAccount(object) :
    
    def __init__(self,tr) :
        
        if tr.event == 'issue' :
            self.account_no = an.Account_Nubmer.get_AN()
        else :
            self.account_no = tr.account_no
        self.amount = 0
        self.dalist_no =0
        self.dalist = DAList(self.account_no,self.get_dalist(),tr.event,tr.amount)
            

    def get_account(self) :
        return (self.account_no,self.amount)
        
    def get_dalist(self) :
        self.dalist_no +=1
        return self.dalist_no
            
    def set_account(self,tr) :
        
        if tr.event == "issue" :
            pass
        else : 
            self.dalist = DAList(self.account_no,self.get_dalist(),tr.event,tr.amount)
        
        if tr.event == 'debt' :
            self.amount -= tr.amount
        else :
            self.amount += tr.amount
                
class DAList(object) :
    dalist= list()
    def __init__(self,account_no,no,event,amount) :
        self.account_no = account_no
        self.no = no
        self.event = event
        self.amount = amount
        self.dalist.append(self)
    
    def get_DAList(self) :
        v = []
        for ll  in self.dalist :
            v.append(ll.__dict__)
        return v
            

Overwriting bank/Account.py


## 회계 처리

    총계정원장 및 계정원장을 관리

In [22]:
%%writefile bank/general_ledger.py
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 29 16:15:51 2016

@author: 06411
"""

class BankDepositAccount(object) :
    def __init__(self) :
        self.account_code = 1
        self.debtamount =0
        self.creditamount= 0
        
    def get_DepositAccount(self) :
        return (self.account_code,self.debtamount, self.creditamount)
        
    def set_DepositAccout(self, tr) :
        print('tr_event', tr.event)
        if tr.event == "debt" :
            self.debtamount += tr.amount
        else :
            self.creditamount += tr.amount
            
class GeneralLedger(object) :
    def __init__(self) :
        self.GL_account =1
        self.GL_debt =0
        self.GL_credit =0
        
    def get_GL(self) :
        return (self.GL_account, self.GL_debt, self.GL_credit)
        
    def set_GL(self,tr) :
        
        if tr.event == "debt" :
            self.GL_debt += tr.amount
        else :
            self.GL_credit += tr.amount

Writing bank/general_ledger.py


##  거래 발생 처리하기

     은행 거래에 대한 발생 기준을 만들고 조회

In [25]:
%%writefile bank/transaction.py
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 29 16:18:08 2016

@author: 06411
"""

class Transaction(object) :
    def __init__(self,tr_no,event,account_no,amount):
        self.tr_no = tr_no
        #event :debt, credit,issue
        self.account_no = account_no
        self.event = event
       
        self.amount = amount
       
    def get_Transaction(self) :
        return (self.tr_no, self.event, self.account_no, self.amount)
        

Writing bank/transaction.py


## 실제 모든 거래를 처리하는 main

    실제 은행 업무 처리하기
    

In [37]:
%%writefile bank/main.py

# -*- coding: utf-8 -*-
"""
Created on Mon Feb 29 16:36:27 2016

@author: 06411
"""

import transaction as ttt
import general_ledger as ggg
import Account as ddd
            
print(" create bank account  ")
ba = ggg.BankDepositAccount()
gl = ggg.GeneralLedger()
    
print(" issue deposit account ")
tr0 = ttt.Transaction(1,"issue",0,100)

print(" issue customer account ")
da = ddd.DepositAccount(tr0)
tr0.account_no = da.account_no

da.set_account(tr0)
ba.set_DepositAccout(tr0)
gl.set_GL(tr0)
print(" inquery customer account :",da.get_account())
print(" inquery deposit account  :",ba.get_DepositAccount())
print(" inquery general ledager  :",gl.get_GL())


print(" credit deposit account ")
tr1 = ttt.Transaction(1,"credit",1,100)
da.set_account(tr1)
ba.set_DepositAccout(tr1)
gl.set_GL(tr1)

print(" inquery customer account :",da.get_account())
print(" inquery deposit account  :",ba.get_DepositAccount())
print(" inquery general ledager  :",gl.get_GL())


print(" debt deposit accoutn ")
print(" debt deposit account ")

tr2 = ttt.Transaction(1,"debt",1,100)
da.set_account(tr2)
ba.set_DepositAccout(tr2)
gl.set_GL(tr2)
print(" inquery customer account :",da.get_account())
print(" inquery deposit account  :",ba.get_DepositAccount())
print(" inquery general ledager  :",gl.get_GL())

print(" DAlist ", da.dalist.get_DAList())

Overwriting bank/main.py


In [52]:
!python bank/main.py

 create bank account  
 issue deposit account 
 issue customer account 
('tr_event', 'issue')
(' inquery customer account :', (1, 100))
(' inquery deposit account  :', (1, 0, 100))
(' inquery general ledager  :', (1, 0, 100))
 credit deposit account 
('tr_event', 'credit')
(' inquery customer account :', (1, 200))
(' inquery deposit account  :', (1, 0, 200))
(' inquery general ledager  :', (1, 0, 200))
 debt deposit accoutn 
 debt deposit account 
('tr_event', 'debt')
(' inquery customer account :', (1, 100))
(' inquery deposit account  :', (1, 100, 200))
(' inquery general ledager  :', (1, 100, 200))
(' DAlist ', [{'amount': 100, 'no': 1, 'event': 'issue', 'account_no': 1}, {'amount': 100, 'no': 2, 'event': 'credit', 'account_no': 1}, {'amount': 100, 'no': 3, 'event': 'debt', 'account_no': 1}])
